In [ ]:
from heapq import heappush, heappop
from collections import defaultdict

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Definir pacientes simulados
pacientes_simulados = [
    {"PacienteID": 1001, "Servicios": ["LABORATORIO", "MASTOGRAFIA", "DENSITOMETRIA", "ULTRASONIDO", "PAPANICOLAOU"]},
    {"PacienteID": 1002, "Servicios": ["RAYOS X", "DENSITOMETRIA", 'LABORATORIO', "ULTRASONIDO"]},
    {"PacienteID": 1003, "Servicios": ["MASTOGRAFIA", "NUTRICION", 'DENSITOMETRIA', "PAPANICOLAOU", "RAYOS X", "LABORATORIO", "ELECTROCARDIOGRAMA"]},
    {"PacienteID": 1004, "Servicios": ["TOMOGRAFIA", "RESONANCIA MAGNETICA", 'LABORATORIO', "LABORATORIO", "DENSITOMETRIA"]},
    {"PacienteID": 1005, "Servicios": ["LABORATORIO", "ELECTROCARDIOGRAMA", "ULTRASONIDO", "DENSITOMETRIA"]},
    {"PacienteID": 1006, "Servicios": ["MASTOGRAFIA", "RAYOS X", "DENSITOMETRIA", "ULTRASONIDO", "PAPANICOLAOU", "LABORATORIO", "ELECTROCARDIOGRAMA"]}
]

# Definir duración promedio (promedio para COYOACAN)
duration_map_live = {
    'DENSITOMETRIA': 7.10,
    'ELECTROCARDIOGRAMA': 6.78,
    'LABORATORIO': 3.23,
    'MASTOGRAFIA': 8.28,
    'NUTRICION': 12.45,
    'PAPANICOLAOU': 7.74,
    'RAYOS X': 5.14,
    'RESONANCIA MAGNETICA': 34.05,
    'TOMOGRAFIA': 18.95,
    'ULTRASONIDO': 14.54
}

# Iniciar estado de salas (una sala por servicio)
machine_state_event = {
    mod: [pd.Timestamp("2023-01-01 08:00:00")] for mod in duration_map_live
}

# Simulación basada en eventos
event_queue = []
simulation_results = []
entry_base = pd.Timestamp("2023-01-01 08:00:00")

# Agregar llegada de pacientes al evento inicial
for i, paciente in enumerate(pacientes_simulados):
    arrival_time = entry_base + pd.Timedelta(minutes=5 * i)
    heappush(event_queue, (arrival_time, "arrival", paciente["PacienteID"], paciente["Servicios"]))

# Estado actual de cada paciente (para saber qué servicio sigue)
paciente_estado = {}

while event_queue:
    current_time, event_type, paciente_id, data = heappop(event_queue)

    if event_type == "arrival":
        paciente_estado[paciente_id] = data
        heappush(event_queue, (current_time, "service", paciente_id, data[0]))

    elif event_type == "service":
        servicio = data
        sala_times = machine_state_event[servicio]
        sala_idx = min(range(len(sala_times)), key=lambda i: sala_times[i])
        inicio_servicio = max(current_time, sala_times[sala_idx])

        # REEMPLAZAR POR TIEMPO REAL
        fin_servicio = inicio_servicio + pd.Timedelta(minutes=duration_map_live[servicio])

        # Actualizar estado de sala
        machine_state_event[servicio][sala_idx] = fin_servicio

        # Guardar resultado
        simulation_results.append({
            "PacienteID": paciente_id,
            "Servicio": servicio,
            "Sala": f"{servicio}_Sala{sala_idx + 1}",
            "Inicio": inicio_servicio.time(),
            "Fin": fin_servicio.time()
        })

        # Programar siguiente servicio si hay más
        servicios_restantes = paciente_estado[paciente_id]
        if servicio in servicios_restantes:
            servicios_restantes.remove(servicio)
        if servicios_restantes:
            heappush(event_queue, (fin_servicio + pd.Timedelta(minutes=2), "service", paciente_id, servicios_restantes[0]))

# Mostrar resultados
df_simulacion_eventos = pd.DataFrame(simulation_results)

In [ ]:
df_simulacion_eventos

,PacienteID,Servicio,Sala,Inicio,Fin
0,1001,LABORATORIO,LABORATORIO_Sala1,08:00:00,08:03:13.800000
1,1002,RAYOS X,RAYOS X_Sala1,08:05:00,08:10:08.400000
2,1001,MASTOGRAFIA,MASTOGRAFIA_Sala1,08:05:13.800000,08:13:30.599999
3,1003,MASTOGRAFIA,MASTOGRAFIA_Sala1,08:13:30.599999,08:21:47.399999
4,1002,DENSITOMETRIA,DENSITOMETRIA_Sala1,08:12:08.400000,08:19:14.400000
5,1004,TOMOGRAFIA,TOMOGRAFIA_Sala1,08:15:00,08:33:57
6,1001,DENSITOMETRIA,DENSITOMETRIA_Sala1,08:19:14.400000,08:26:20.400000
7,1005,LABORATORIO,LABORATORIO_Sala1,08:20:00,08:23:13.800000
8,1002,LABORATORIO,LABORATORIO_Sala1,08:23:13.800000,08:26:27.600000
9,1003,NUTRICION,NUTRICION_Sala1,08:23:47.399999,08:36:14.399999
